In [1]:
from scrapy.selector import Selector
from tqdm import tqdm

In [2]:
!chromedriver --version

ChromeDriver 119.0.6045.105 (38c72552c5e15ba9b3117c0967a0fd105072d7c6-refs/branch-heads/6045@{#1103})


In [3]:
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('headless')
# Set the path to the ChromeDriver executable
chrome_driver_path = 'C:/Program Files/chromedriver-win64/chromedriver-win64/chromedriver.exe'

# Initialize the Chrome driver with the specified path
driver = webdriver.Chrome(executable_path=chrome_driver_path)

url = "https://www.imdb.com/title/tt1517268/reviews/?ref_=tt_ov_rt"
driver.get(url)
time.sleep(1)
print(driver.title)
time.sleep(1)
body = driver.find_element(By.CSS_SELECTOR, 'body')
body.send_keys(Keys.PAGE_DOWN) ### scroll down 
time.sleep(1)
body.send_keys(Keys.PAGE_DOWN)
time.sleep(1)
body.send_keys(Keys.PAGE_DOWN)

C:\Users\mariem\AppData\Local\Temp\ipykernel_2668\1500758899.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_driver_path)


Barbie (2023) - Barbie (2023) - User Reviews - IMDb


In [4]:
sel = Selector(text = driver.page_source)
review_counts = sel.css('.lister .header span::text').extract_first().replace(',','').split(' ')[0]
#to extract the text content of all the span elements that are descendants of an element with class .header that is itself a descendant of an element with class .lister. The text content of the first matching element is extracted.
review_counts = review_counts.replace("\xa0", "")
more_review_pages = int(int(review_counts)/25)

In [5]:
reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container') ## get all the loaded reviews
len(reviews)


25

In [6]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


css_selector = 'load-more-trigger'

# Loop through the pages
for i in range(more_review_pages):
    try:
        # Use WebDriverWait to wait for the button to be clickable
        load_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, css_selector))
        )
        load_more_button.click()
        time.sleep(2.5)  # Adjust this if needed
    except Exception as e:
        print(f"Error clicking 'Load More' button: {e}")
        # Retry the operation after a short delay
        time.sleep(2)


In [9]:
import pandas as pd
review_list = []
rating_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container') ## get all the loaded reviews

for d in tqdm(reviews):
    try:
        sel2 = Selector(text = d.get_attribute('innerHTML'))
        try:
            rating = sel2.css('.rating-other-user-rating span::text').extract_first()
        except:
            rating = np.NaN
        try:
            review = sel2.css('.text.show-more__control::text').get()
        except:
            review = np.NaN
        
        rating_list.append(rating)
        review_list.append(review)
    except Exception as e:
        error_url_list.append(url)
        error_msg_list.append(e)
    
review_df = pd.DataFrame({  
    'Rating':rating_list, 
    'Review':review_list
    })


C:\Users\mariem\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\mariem\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\mariem\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\mariem\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
100%|█████████████████████████████████████████████████████████████████████████████| 1539/1539 [00:10<00:00, 141.41it/s]


In [10]:
len(review_list)

1539

In [11]:
df = review_df.copy()

# Sentiment analysis

In [12]:
df

,Rating,Review
0,6,"Margot does the best with what she's given, bu..."
1,6,"Before making Barbie (2023),"
2,7,The first reason to go see it:
3,10,As much as it pains me to give a movie called ...
4,6,"As a woman that grew up with Barbie, I was ver..."
...,...,...
1534,5,This is my first review at imdb and tbh I real...
1535,5,"I'm NOT a Barbie girl in NOT the Barbie World,..."
1536,3,The movie is mediocre at best. I expected an o...
1537,5,Some things were super clever and made me laug...


dropping empty ratings and reviews

In [13]:
df =df.dropna()
df.reset_index(inplace=True)

In [14]:
df =df.drop('index' , axis =1)

# Calculating sentiment score for preprocessing

In [15]:
!pip install vaderSentiment


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 23.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid_obj= SentimentIntensityAnalyzer()

In [17]:

####on the whole review
df = df.assign(Polarity=None)


for i in range(len(df)) : 
   
    scores = sid_obj.polarity_scores(df["Review"][i])
    df["Polarity"][i] = scores


In [18]:
df = df.assign(Compound=None)
for i in range(len(df)):
    df["Compound"][i] = df["Polarity"][i]["compound"]

A compound score of: </br>
            1. Positive (greater than 0.05): indicates a positive sentiment </br>
            2. Neutral (between -0.05 and 0.05): indicates a neutral sentiment </br>
            3. Negative (less than -0.05): indicates a negative sentiment </br>

In [19]:
df = df.assign(Sentiment=None)

for i in   range(len(df)) :
    if df["Compound"][i] >= 0.05 : 
        value = "+1"
    if df["Compound"][i] <= -0.05 :
        value = "-1"
    if df["Compound"][i]>-00.5 and df["Compound"][i] < 0.05:
        value  ="0"
        
    df["Sentiment"][i] = value
    

In [20]:
df

,Rating,Review,Polarity,Compound,Sentiment
0,6,"Margot does the best with what she's given, bu...","{'neg': 0.121, 'neu': 0.665, 'pos': 0.214, 'co...",0.957,+1
1,6,"Before making Barbie (2023),","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,0
2,7,The first reason to go see it:,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,0
3,10,As much as it pains me to give a movie called ...,"{'neg': 0.058, 'neu': 0.769, 'pos': 0.173, 'co...",0.9883,+1
4,6,"As a woman that grew up with Barbie, I was ver...","{'neg': 0.051, 'neu': 0.895, 'pos': 0.053, 'co...",-0.2247,0
...,...,...,...,...,...
1513,5,This is my first review at imdb and tbh I real...,"{'neg': 0.093, 'neu': 0.886, 'pos': 0.021, 'co...",-0.8902,-1
1514,5,"I'm NOT a Barbie girl in NOT the Barbie World,...","{'neg': 0.064, 'neu': 0.852, 'pos': 0.084, 'co...",0.5812,+1
1515,3,The movie is mediocre at best. I expected an o...,"{'neg': 0.083, 'neu': 0.765, 'pos': 0.152, 'co...",0.9161,+1
1516,5,Some things were super clever and made me laug...,"{'neg': 0.14, 'neu': 0.63, 'pos': 0.23, 'compo...",0.8873,+1


VADER is a pre-built sentiment analysis tool that is trained on social media data and is known for its performance on short, informal texts. However, it may not always be suitable for all types of text, especially when dealing with sarcasm or nuanced sentiments.

it's not uncommon for sentiment analysis models to struggle with sarcasm, as they often rely on patterns and associations in the training data. Sarcasm can be tricky because the words used may have a positive sentiment individually, but the overall context may convey a negative sentiment. and in our data we have a lot of sarcastic reviews.

the literal meaning of the words might be positive, but the actual sentiment conveyed is negative or vice versa. This can lead to discrepancies between the calculated sentiment scores and the true sentiment of the text.

So what we did is compare the attributed sentiment from the compound score with the rating, if the two values don't match then we consider the sentiment attributed to be wrong.

In [21]:
ind=[]
for i in range(len(df)) : 
    if int(df["Rating"][i]) in range(0,3) and df["Sentiment"][i] !="-1": 
        ind.append(i)
    if int(df["Rating"][i]) in range(4,6) and  df["Sentiment"][i] !="0" :
        ind.append(i)
    if int(df["Rating"][i]) in range(7,10)and  df["Sentiment"][i] !="+1" :
        ind.append(i)
        
        

In [ ]:
dff = df.drop(ind).reset_index(drop=True, level=0)

dff =dff.drop('index' , axis =1)

In [24]:
dff

,Rating,Review,Polarity,Compound,Sentiment
0,6,"Margot does the best with what she's given, bu...","{'neg': 0.121, 'neu': 0.665, 'pos': 0.214, 'co...",0.957,+1
1,6,"Before making Barbie (2023),","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,0
2,10,As much as it pains me to give a movie called ...,"{'neg': 0.058, 'neu': 0.769, 'pos': 0.173, 'co...",0.9883,+1
3,6,"As a woman that grew up with Barbie, I was ver...","{'neg': 0.051, 'neu': 0.895, 'pos': 0.053, 'co...",-0.2247,0
4,8,"It pains me to say it, but I enjoyed this movi...","{'neg': 0.042, 'neu': 0.591, 'pos': 0.367, 'co...",0.9505,+1
...,...,...,...,...,...
1155,5,***SPOILERS**,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,0
1156,4,While the Barbie movie delivers a range of col...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,0
1157,1,Too many sexual references and doesn't grasp t...,"{'neg': 0.069, 'neu': 0.883, 'pos': 0.049, 'co...",-0.5793,-1
1158,3,This Barbie movie is a first for me. The first...,"{'neg': 0.114, 'neu': 0.812, 'pos': 0.073, 'co...",-0.9404,-1


Since we are going to use this data to validate our model later it needs to match the IMDB Dataset

In [25]:
threshold = 5
dff['Rating'] = dff['Rating'].astype(int)  # Convert 'Rating' to integers if needed
dff['Sentiment'] = dff['Rating'].apply(lambda x: '1' if x > threshold else '0')


In [26]:
dff

,Rating,Review,Polarity,Compound,Sentiment
0,6,"Margot does the best with what she's given, bu...","{'neg': 0.121, 'neu': 0.665, 'pos': 0.214, 'co...",0.957,1
1,6,"Before making Barbie (2023),","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,1
2,10,As much as it pains me to give a movie called ...,"{'neg': 0.058, 'neu': 0.769, 'pos': 0.173, 'co...",0.9883,1
3,6,"As a woman that grew up with Barbie, I was ver...","{'neg': 0.051, 'neu': 0.895, 'pos': 0.053, 'co...",-0.2247,1
4,8,"It pains me to say it, but I enjoyed this movi...","{'neg': 0.042, 'neu': 0.591, 'pos': 0.367, 'co...",0.9505,1
...,...,...,...,...,...
1155,5,***SPOILERS**,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,0
1156,4,While the Barbie movie delivers a range of col...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,0
1157,1,Too many sexual references and doesn't grasp t...,"{'neg': 0.069, 'neu': 0.883, 'pos': 0.049, 'co...",-0.5793,0
1158,3,This Barbie movie is a first for me. The first...,"{'neg': 0.114, 'neu': 0.812, 'pos': 0.073, 'co...",-0.9404,0


In [27]:
selected_columns = ['Review', 'Sentiment']
df_selected = dff[selected_columns]


In [28]:
df_shuffled = df_selected.sample(frac=1, random_state=42).reset_index(drop=True)
df_shuffled

,Review,Sentiment
0,"The hype, enthusiasm, technology, acting skill...",0
1,I went into this film as part of a date and ha...,0
2,"Such a silly, beautiful, wonderful movie! This...",1
3,This movie was so unexpected.,1
4,"I laughed, I felt understood, I felt touched. ...",1
...,...,...
1155,I was really looking forward this movie and my...,1
1156,"At first, I didn't want to go on this film. Th...",0
1157,Title: A Perfect 10: Barbie's Spectacular Movi...,1
1158,"""Barbie"" may seem like an improbable concept f...",1


In [29]:
import pandas as pd

df_shuffled.to_csv('imdb_val.csv', index=False)
